# NIfTI Conversion Script for Sarcopenia Data Descriptor

Convert .tag files and DICOM images into nifti format for use in imaging research. Pre-print can be found online at: INSERT LINK.

Last updated by Kareem Wahid, Jan 14, 2022.

Code assumes you have the following directory structure:

In [ ]:
"""
Master_folder/
├── nifti_conversion_github.ipynb
├── input/ <- this contains all the master DICOM image folders (direct download from TCIA)
│   ├──Patient ID folders 
│      ├──Date folders 
│         ├──Series folders
│            ├──DICOM files (.dcm)
├── tags/ <- segmentation data provided for you 
│   ├──tag files (c3 ID.dcm.tag)
├── dicoms/ <- 2D DICOM data provided for you 
│   ├──DICOM files (c3 ID.dcm)
"""
;

## Dependencies

In [1]:
import pandas as pd
import os
import shutil
from DicomRTTool.ReaderWriter import DicomReaderWriter # pip install DicomRTTool
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import sys
import glob

## Helper Fuctions

In [ ]:
def write_dicomrt_to_nifti(dicom_pt_path, output_folder):
    '''
    Read DICOM image and transform to numpy array format and corresponding sitk object. Also writes image to folder. 

            Parameters:
                    dicom_pt_path (str): Path to input DICOM folder.
                    output_folder (str): Path to output folder.

            Returns:
                    image (ndarray): 3D numpy array of image.
                    dicom_sitk_handle (sitk obj): simpleitk object that conatins DICOM metadata. 
    '''
    
    Dicom_reader = DicomReaderWriter(description='Examples', series_instances_dictionary = {}) # instantiate

    Dicom_reader.walk_through_folders(dicom_pt_path)

    all_rois = Dicom_reader.return_rois(print_rois=False)
    
    Dicom_reader.get_images()
    image = Dicom_reader.ArrayDicom # image array
    dicom_sitk_handle = Dicom_reader.dicom_handle # SimpleITK image handle
    
    sitk.WriteImage(dicom_sitk_handle, os.path.join(output_folder, 'Image.nii.gz')) # write the image to the output folder
            
    return image, dicom_sitk_handle # need these for later 
            
                   
def read_tag_to_array(tag_file_path, image): # image is array
    '''
    Convert tag segmentation file to 2D numpy array. 

            Parameters:
                    tag_file_path (str): Path to .tag file, i.e., segmentation.
                    image (ndarray): 3D numpy array of image.

            Returns:
                    d1image_final (ndarray): 2D numpy array of segmentation.
    '''
    
    input_size = image[1,:,:].flatten().shape
    
    dtype = np.dtype('B')
    with open(tag_file_path, "rb") as f:
        numpy_data = np.fromfile(f,dtype)
        
    to_trim = numpy_data.shape[0] - input_size[0]
    
    trimmed = numpy_data[-1:to_trim-1:-1]
    
    d1image = np.reshape(trimmed, image[1,:,:].shape) # 512x512 for these CT cases
    
    d1image_final = np.flip(d1image) # images are flipped so this fixes that 
    
    return d1image_final


def insert_location_to_tag_array(image, d1image_final, slice_level_file_path, dicom_image_folder):
    '''
    Convert 2D segmentation array to 3D numpy array. Some code adapted from https://stackoverflow.com/questions/59458801/how-to-sort-dicom-slices-in-correct-order.

            Parameters:
                    image (ndarray): 3D numpy array of image.
                    d1image_final (ndarray): 2D numpy array of segmentation.
                    slice_level_file_path (str): Path to 2D DICOM slice. 
                    dicom_image_folder (str): Path to CT DICOM folder, since the TCIA download returns multiple folders. 
                    
            Returns:
                    fat_muscle_array (ndarray): 3D numpy array of segmentation.
    '''
    
    ds = pydicom.read_file(slice_level_file_path)
    location_of_interest = ds.SliceLocation
    
    # skip files with no SliceLocation (eg scout views)
    slices = []
    skipcount = 0
    for f in [os.path.join(dicom_image_folder, file) for file in os.listdir(dicom_image_folder)]:
        ds = pydicom.read_file(f)
        #print(f)
        if hasattr(ds, 'SliceLocation'):
            slices.append(ds)
        else:
            skipcount = skipcount + 1

    # ensure they are in the correct order
    slices = sorted(slices, key=lambda s: s.SliceLocation)
    
    for number, slice_ in enumerate(slices):
        if slice_.SliceLocation == location_of_interest:
            array_slice_location = number
    
    fat_muscle_array = np.zeros(image.shape)
    
    fat_muscle_array[array_slice_location, :, :] = d1image_final # fill the slice location with the 2d mask data
    
    return fat_muscle_array


def seperate_and_save_array(fat_muscle_array, dicom_sitk_handle, output_folder):
    '''
    Split 3D segmentation array into adipose and skeletal muscle components. Output these as binary mask nifti files to folder.

            Parameters:
                    fat_muscle_array (ndarray): 3D numpy array of segmentation.
                    dicom_sitk_handle (sitk obj): simpleitk object that conatins DICOM metadata. 
                    output_folder (str): Path to output folder.
                    
            Returns:
                    None, writes nifti files to folder. 
    '''
    
    fat_array = fat_muscle_array.copy() # make copy to avoid errors later 
    muscle_array = fat_muscle_array.copy()
    
    fat_array[fat_array != 5] = 0
    muscle_array[muscle_array != 1] = 0
    
    fat_array_sitk = sitk.GetImageFromArray(fat_array)
    fat_array_sitk.CopyInformation(dicom_sitk_handle)

    muscle_array_sitk = sitk.GetImageFromArray(muscle_array)
    muscle_array_sitk.CopyInformation(dicom_sitk_handle)

    sitk.WriteImage(fat_array_sitk, os.path.join(output_folder, 'fat.nii.gz'))
    sitk.WriteImage(muscle_array_sitk, os.path.join(output_folder, 'muscle.nii.gz'))

## Main Function

In [ ]:
def TCIA_tag_to_nifti(dicom_pt_path, tag_path, slice_level_path, output_path):
    
    MRN_full = os.path.split(dicom_pt_path)[-1]
    MRN_number = MRN_full.split('-0')[-1]
    
    output_folder = os.path.join(output_path, MRN_full) # this is where all the files will go
    
    tag_MRNs = [file.split(' ')[1].split('.')[0] for file in os.listdir(tag_path)] # need this to generate corresponding folders
    
    if not os.path.exists(output_folder) and MRN_number in tag_MRNs: # edited this, should lead to 396 folders
        os.makedirs(output_folder)
    
    slice_level_file_path = os.path.join(slice_level_path, 'c3 ' + MRN_number +'.dcm')
    
    if os.path.isfile(slice_level_file_path) == False:
        print("No c3 segmentation for " + str(MRN_number))
        return 
    
    tag_file_path = os.path.join(tag_path, 'c3 ' + MRN_number +'.dcm.tag')
    
    for root, dirs, files in os.walk(dicom_pt_path): # find the folder with the dicom image since there are multiple folders (series)
        for d in dirs:
            d_path = os.path.join(root,d)
            if len(os.listdir(d_path)) > 10:
                dicom_image_folder = d_path
    
    # run functions here
    
    image, dicom_sitk_handle = write_dicomrt_to_nifti(dicom_pt_path, output_folder) # writes files and returns image array for later
    
    d1image_final = read_tag_to_array(tag_file_path, image, output_folder)
    
    fat_muscle_array = insert_location_to_tag_array(image, d1image_final, slice_level_file_path, dicom_image_folder)
    
    seperate_and_save_array(fat_muscle_array, dicom_sitk_handle, output_folder) # writes to .nii.gz files

## Applying functions to data

The input_path will contain all the folders of the DICOM data downloaded directly from TCIA, i.e., HNSCC-01-0240, HNSCC-01-0320, etc.. 

The tag_path and slice_level_path will are the folders containing outputs from sliceOmatic.

The output path is an empty directory where the new patient folders containing the corresponding nifti files will be stored.

In [ ]:
%%time

input_path = "input" # this contains all the master DICOM image folders (direct download from TCIA)
tag_path = "tags" # given 
slice_level_path = "dicoms" # given
output_path = "output" # generated

# make output_directory if it does not exist
if not os.path.exists(output_path):
    os.makedirs(output_path)

dicom_folders = os.listdir(input_path)
for folder in dicom_folders: # iterate over all folders
    dicom_pt_path = os.path.join(input_path, folder)
    TCIA_tag_to_nifti(dicom_pt_path, tag_path, slice_level_path, output_path) # run 